# 2D Range Tree Construction

**Range Tree in $\mathbb{R}^2$:**
- Primary tree: **BST on x-coordinates** (leaves store points sorted by x)
- Each internal node $v$ has an **auxiliary tree $T(v)$**: BST on y-coordinates of all descendant leaves of $v$
- Construction visualized step-by-step via animation

In [12]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.animation import FuncAnimation
from matplotlib.lines import Line2D
from IPython.display import HTML
import warnings
warnings.filterwarnings('ignore')
np.random.seed(42)

In [13]:
# Generate random 2D points
def generate_random_points(n, x_range=(0, 100), y_range=(0, 100)):
    points = []
    for _ in range(n):
        x = round(np.random.uniform(*x_range), 1)
        y = round(np.random.uniform(*y_range), 1)
        points.append((x, y))
    return points

In [14]:
# ── 2D Range Tree Data Structure ──────────────────────────────────

class AuxYNode:
    """BST node for the auxiliary tree T(v), sorted by y-coordinate."""
    def __init__(self, points):
        self.points = points          # all points stored at this subtree
        self.left = None
        self.right = None
        self.split_y = None
        if len(points) > 1:
            pts_sorted = sorted(points, key=lambda p: p[1])
            mid = len(pts_sorted) // 2
            self.split_y = pts_sorted[mid][1]
            self.left  = AuxYNode(pts_sorted[:mid])
            self.right = AuxYNode(pts_sorted[mid:])

class RangeTreeNode:
    """Primary BST node, split on x-coordinate.
       Leaves store single points.
       Each node v stores T(v): auxiliary BST on y-coords of all descendant leaves."""
    def __init__(self, points):
        self.points = points          # all points in this subtree
        self.left = None
        self.right = None
        self.split_x = None
        self.aux_tree = AuxYNode(points)   # T(v)

        if len(points) > 1:
            pts_sorted = sorted(points, key=lambda p: p[0])
            mid = len(pts_sorted) // 2
            self.split_x = pts_sorted[mid][0]
            self.left  = RangeTreeNode(pts_sorted[:mid])
            self.right = RangeTreeNode(pts_sorted[mid:])

class RangeTree2D:
    def __init__(self, points):
        self.root = RangeTreeNode(points)

print("2D Range Tree structure defined ✓")

2D Range Tree structure defined ✓


In [15]:
# ── Helpers: collect construction steps ────────────────────────────

def collect_build_steps(node, depth=0, x_lo=0, x_hi=100):
    """DFS traversal that records each node processed during construction.
       Each step = (points, split_x, depth, x_lo, x_hi, aux_tree)"""
    steps = []
    if node is None:
        return steps
    steps.append(dict(
        points=list(node.points),
        split_x=node.split_x,
        depth=depth,
        x_lo=x_lo,
        x_hi=x_hi,
        aux_tree=node.aux_tree,
        is_leaf=(node.left is None and node.right is None),
    ))
    if node.left:
        steps += collect_build_steps(node.left, depth+1, x_lo,
                                     node.split_x if node.split_x else x_hi)
    if node.right:
        steps += collect_build_steps(node.right, depth+1,
                                     node.split_x if node.split_x else x_lo, x_hi)
    return steps


def collect_aux_y_levels(aux_node, depth=0):
    """Return list of (split_y, depth) for drawing the auxiliary y-BST."""
    levels = []
    if aux_node is None:
        return levels
    levels.append(dict(
        split_y=aux_node.split_y,
        depth=depth,
        points=list(aux_node.points),
        is_leaf=(aux_node.left is None and aux_node.right is None),
    ))
    if aux_node.left:
        levels += collect_aux_y_levels(aux_node.left, depth+1)
    if aux_node.right:
        levels += collect_aux_y_levels(aux_node.right, depth+1)
    return levels

print("Step collection helpers defined ✓")

Step collection helpers defined ✓


In [16]:
# ── Animated Construction of 2D Range Tree ──────────────────────

def animate_range_tree(points):
    tree = RangeTree2D(points)
    steps = collect_build_steps(tree.root)
    n_steps = len(steps)
    all_x = [p[0] for p in points]
    all_y = [p[1] for p in points]
    pad = 5
    x_min_g, x_max_g = min(all_x) - pad, max(all_x) + pad
    y_min_g, y_max_g = min(all_y) - pad, max(all_y) + pad

    # ── Figure: left = 2D space, right = auxiliary T(v) diagram ──
    fig, (ax_space, ax_aux) = plt.subplots(1, 2, figsize=(15, 7),
                                            gridspec_kw={'width_ratios': [1.2, 1]})
    fig.suptitle("2D Range Tree Construction", fontsize=14, fontweight='bold')

    # Color palette per depth
    depth_colors = plt.cm.Set1(np.linspace(0, 1, 9))

    # ── persistent artists ──
    # plot all points faintly at start
    ax_space.scatter(all_x, all_y, c='lightgray', s=40, zorder=1, edgecolors='gray', linewidths=0.5)
    ax_space.set_xlim(x_min_g, x_max_g)
    ax_space.set_ylim(y_min_g, y_max_g)
    ax_space.set_xlabel('x')
    ax_space.set_ylabel('y')
    ax_space.set_title('Primary BST on x-coordinate')

    ax_aux.set_title('Auxiliary tree T(v) — BST on y')
    ax_aux.set_xlim(-1, 1)
    ax_aux.set_ylim(y_min_g, y_max_g)
    ax_aux.set_xlabel('')
    ax_aux.set_ylabel('y')

    def draw_aux_tree(ax, aux_node, y_lo, y_hi, x_center=0, x_half=0.45, depth=0):
        """Recursively draw the y-BST in the right panel."""
        if aux_node is None:
            return
        color = depth_colors[min(depth, len(depth_colors)-1)]
        if aux_node.split_y is not None:
            # draw horizontal split line
            ax.plot([x_center - x_half, x_center + x_half],
                    [aux_node.split_y, aux_node.split_y],
                    color=color, linewidth=1.5, linestyle='--', alpha=0.7)
            ax.text(x_center + x_half + 0.02, aux_node.split_y,
                    f'y={aux_node.split_y:.1f}', fontsize=7, color=color,
                    va='center', alpha=0.8)
        # draw points
        for p in aux_node.points:
            ax.plot(x_center, p[1], 'o', color=color, markersize=5, alpha=0.6)
        if aux_node.left:
            draw_aux_tree(ax, aux_node.left, y_lo, aux_node.split_y,
                          x_center - x_half/2, x_half/2, depth+1)
        if aux_node.right:
            draw_aux_tree(ax, aux_node.right, aux_node.split_y, y_hi,
                          x_center + x_half/2, x_half/2, depth+1)

    def update(frame):
        step = steps[frame]
        pts = step['points']
        split_x = step['split_x']
        depth = step['depth']
        x_lo = step['x_lo']
        x_hi = step['x_hi']
        aux = step['aux_tree']
        is_leaf = step['is_leaf']
        color = depth_colors[min(depth, len(depth_colors)-1)]

        # ── Left panel: highlight points + draw x-split line ──
        px = [p[0] for p in pts]
        py = [p[1] for p in pts]
        ax_space.scatter(px, py, c=[color], s=70, zorder=3,
                         edgecolors='black', linewidths=0.8)

        if split_x is not None:
            # vertical split line confined to this node's x-range
            ax_space.plot([split_x, split_x], [y_min_g, y_max_g],
                          color=color, linewidth=1.5, linestyle='--', alpha=0.6)
            ax_space.text(split_x, y_max_g + 1,
                          f'x={split_x:.1f}  d={depth}',
                          fontsize=7, ha='center', color=color, fontweight='bold')

        # Shade the x-region
        rect = patches.Rectangle((x_lo, y_min_g), x_hi - x_lo, y_max_g - y_min_g,
                                  linewidth=0, facecolor=color, alpha=0.04)
        ax_space.add_patch(rect)

        # ── Right panel: show T(v) for this node ──
        ax_aux.cla()
        ax_aux.set_title(f'T(v) at depth {depth}  |  {len(pts)} pts', fontsize=11)
        ax_aux.set_xlim(-1, 1)
        ax_aux.set_ylim(y_min_g, y_max_g)
        ax_aux.set_ylabel('y')
        # draw faint grid
        ax_aux.axvline(0, color='lightgray', linewidth=0.5)
        draw_aux_tree(ax_aux, aux, y_min_g, y_max_g)

        # ── suptitle update ──
        fig.suptitle(f"2D Range Tree Construction — step {frame+1}/{n_steps}  "
                     f"(depth={depth}, {'leaf' if is_leaf else 'internal'})",
                     fontsize=13, fontweight='bold')

    anim = FuncAnimation(fig, update, frames=n_steps,
                         interval=1200, repeat=False)
    plt.close(fig)
    return HTML(anim.to_jshtml())

# ── Run ──
pts = generate_random_points(12)
print("Points:", pts)
animate_range_tree(pts)

Points: [(37.5, 95.1), (73.2, 59.9), (15.6, 15.6), (5.8, 86.6), (60.1, 70.8), (2.1, 97.0), (83.2, 21.2), (18.2, 18.3), (30.4, 52.5), (43.2, 29.1), (61.2, 13.9), (29.2, 36.6)]
